# Data inputs and Display Libraries

In [2]:

import pandas as pd
import numpy as np
import pickle
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Modelling Libraries

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import model_selection

from xgboost import XGBClassifier
import pickle
from sklearn.model_selection import GridSearchCV


# Metrics Libraries


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [5]:
!git clone https://github.com/ragamudra/workshop_trees

Cloning into 'workshop_trees'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [6]:
!unrar x 'workshop_trees/datasets.rar'  'workshop_trees'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from workshop_trees/datasets.rar

Extracting  workshop_trees/y_train1                                        0%  OK 
Extracting  workshop_trees/df_all_test2                                    1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33%  OK 
Extracting  workshop_trees/df_all_train2                                  34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 

In [7]:
fl_out = "./workshop_trees"
def pick_in(obj_name):
    fl_out1 = fl_out + "/" + obj_name
    pickle_in = open(fl_out1,"rb")
    mod1= pickle.load(pickle_in)
    
    return mod1

In [8]:
list_objs = ["df_all_train2","y_train1","df_all_test2","y_test1"]

for i in list_objs:
     globals()[i]= pick_in(i)

In [9]:
def auc1_scr(mod1,test_set,actual1):
  
    mod = eval(mod1)
    pred1=mod.predict_proba(test_set)[:,1]
    fpr, tpr, thresholds = roc_curve(actual1, pred1)
    auc1 = auc(fpr, tpr)
    
    return auc1

In [10]:
ab = AdaBoostClassifier(n_estimators=100, random_state=0)
ab.fit(df_all_train2,y_train1)
###took 49 seconds

AdaBoostClassifier(n_estimators=100, random_state=0)

In [11]:
auc1_te = auc1_scr("ab",df_all_test2,y_test1)
auc1_tr = auc1_scr("ab",df_all_train2,y_train1)

In [12]:
auc1_te,auc1_tr

(0.6239087454925901, 0.6283670987340262)

# Grid Search

In [ ]:
# AB_grid= AdaBoostClassifier(random_state=42)

# params = {
#     'n_estimators': [100,500],
#     'learning_rate': [0.2,0.5,1],
#     'algorithm': ['SAMME','SAMME.R'],
#     'base_estimator' : [DecisionTreeClassifier(max_depth=1),DecisionTreeClassifier(max_depth=2),DecisionTreeClassifier(max_depth=5)]
# }

# grid_search = GridSearchCV(estimator=AB_grid, 
#                            param_grid=params, 
#                            cv=2, n_jobs=5, verbose=1, scoring = "roc_auc")


# grid_search.fit(df_all_test2,y_test1)

In [ ]:
# score_df = pd.DataFrame(grid_search.cv_results_)
# score_df.head()
# score_df.sort_values(["rank_test_score"]).head(5)

# Gradient Boosting

In [13]:
gb = GradientBoostingClassifier(max_depth=5,n_estimators=300, learning_rate=0.5)
gb.fit(df_all_train2,y_train1)


GradientBoostingClassifier(learning_rate=0.5, max_depth=5, n_estimators=300)

In [14]:
auc1_te = auc1_scr("gb",df_all_test2,y_test1)
auc1_tr = auc1_scr("gb",df_all_train2,y_train1)

In [15]:
auc1_te,auc1_tr

(0.9028183253999471, 0.948305573182251)

In [16]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(df_all_train2,y_train1)

XGBClassifier()

In [17]:
auc1_te = auc1_scr("xgb",df_all_test2,y_test1)
auc1_tr = auc1_scr("xgb",df_all_train2,y_train1)

In [18]:
auc1_te,auc1_tr

(0.663948910348972, 0.6733339608819728)

In [25]:
class_weights = [0.1,0.9]

xgb_param = XGBClassifier(n_estimators=300,max_depth= 5,class_weights = class_weights,
                          subsample= 0.2,colsample_bytree= 0.3,random_state=0)


xgb_param.fit(df_all_train2,y_train1)

XGBClassifier(class_weights=[0.1, 0.9], colsample_bytree=0.3, max_depth=5,
              n_estimators=300, subsample=0.2)

In [26]:
auc1_te = auc1_scr("xgb_param",df_all_test2,y_test1)
auc1_tr = auc1_scr("xgb_param",df_all_train2,y_train1)

In [27]:
auc1_te,auc1_tr

(0.7972238724769605, 0.8106333886227974)